In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from math import ceil
from scipy import stats

# Functions

## Exploring measurement time and frequency stability

In [ ]:
def display_time_distribution(df, x):
    """ A function that is designed to display the measurement time distribution using a histogram
    Args:
        1) df - a dataframe that contains a column with measurement time data
        2) x - name of measurement time column
    """
    # Plot the measurement time distribution using seaborn's histogram
    plt.figure(figsize=(15, 6))
    sns.histplot(df[x], bins=60, kde=True, color='blue')
    plt.xlabel('Period, s')
    plt.ylabel('Count')
    plt.title('Measurement Time Distribution')

    plt.show()
    
def get_measurement_time_df(df, column_name='time'):
    """ A function for building a dataframe that contains information about the time of measurement of each reading of the device
    Args:
        1) df - a dataframe that contains a time column
        2) column_name - the name of the column containing time data (in seconds)
    Returns:
        time_measurement_df - dataframe that contains information about the time of measurement of each reading of the device
    """
    period_dict = {'start_time': [], 'end_time': [], 'measurement_time': []}
    prev_time = None
    curr_time = None
    for index, row in df.iterrows():
        if(index == len(df)):
            break
            
        prev_time = curr_time
        curr_time = row[column_name]

        if prev_time != None:
            period_dict['start_time'].append(prev_time)
            period_dict['end_time'].append(curr_time)
            period_dict['measurement_time'].append(curr_time - prev_time)
    
    time_measurement_df = pd.DataFrame.from_dict(period_dict)
    return time_measurement_df


def display_time_distribution(df, x):
    """ A function that is designed to display the measurement time distribution using a histogram
    Args:
        1) df - a dataframe that contains a column with measurement time data
        2) x - name of measurement time column
    """
    # Plot the measurement time distribution using seaborn's histogram
    plt.figure(figsize=(25, 6))
    sns.histplot(df[x], bins=60, kde=True, color='blue')
    plt.xlabel('Period, s')
    plt.ylabel('Count')
    plt.title('Measurement Time Distribution')

    plt.show()


def display_data_collection_stability(df):
    """A function that displays the stability of data collection relative to the time when the measurements were taken and 
       relative to the measurement number
    Args:
        1) df - time_measurement_df
    """
    fig, axes = plt.subplots(2, 1, figsize=(15, 14))
    axes[0].plot(range(len(df[df['lost_data'] == 0])), df[df['lost_data'] == 0]['measurement_time'])
    axes[0].set_title(f'Stability of Data Collection (Filtered)\nAverage frequency: {freq:.3f} Hz')
    axes[0].set_xlabel('number of measurement')
    axes[0].set_ylabel('measurement time')
    
    axes[1].plot(df[df['lost_data'] == 0]['start_time'], df[df['lost_data'] == 0]['measurement_time'])
    axes[1].set_title(f'Stability of Data Collection (Filtered)\nAverage frequency: {freq:.3f} Hz')
    axes[1].set_xlabel('time of measurement')
    axes[1].set_ylabel('measurement time')
    
    plt.show()

## Visualizing results

In [ ]:
def display_three_axes(df, y, x=None, title=None, x_label=None, y_label=None, filename=None):
    """ Function for visualization of three axes (X, Y, Z) of the measurement results of the device
    Args:
        1) df - a dataframe containing the results of the device measurement
        2) x - the name of the column of the dataframe that contains the data for the OX axis (for the three lines, this is the same data)
        3) y - the list containing the dataframe column names corresponding to the OX, OY, and OZ axis measurements, respectively
        4) title - title of the graph
        5) x_label - the name of the OX axis of the graph
        6) y_label - the name of the OY axis of the graph
        7) filename - the relative path where the file will be saved (with the file name, the file extension is not required) or just the filename
    Returns:
        Nothing, but plots graph
    """
    plt.figure(figsize=(12, 6))
    if x is None:
        plt.plot(df[y[0]], label='X')
        plt.plot(df[y[1]], label='Y')
        plt.plot(df[y[2]], label='Z')
    else:
        plt.plot(df[x], df[y[0]], label='X')
        plt.plot(df[x], df[y[1]], label='Y', color='orange')
        plt.plot(df[x], df[y[2]], label='Z', color='green')
           
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.legend()
    if filename:
        plt.savefig(f'{filename}.png', bbox_inches='tight')
    plt.show()

## Data Filtering

In [ ]:
def display_raw_filtered_data_zoom(df, x, y, x_lims, y_lims, title=None, x_label=None, y_label=None, filename=None, zoom_axes=None):
    """ Function to display raw and filtered data on the same graph with scaling to better display the filtering
    Args:
        1) df - a dataframe containing the results of the device measurement
        2) x - the name of the column of the dataframe that contains the data for the OX axis of the graph
        3) y - the list containing the column names of the data frame corresponding to the raw and filtered data, respectively, on a given axis (OX, OY, or OZ)
        4) x_lims - the list of limits along the OX axis for zooming ([zoom_xmin; zoom_xmax])
        5) y_lims - the list of limits along the OY axis for zooming ([zoom_ymin; zoom_ymax])
        6) title - title of the graph
        7) x_label - the name of the OX axis of the graph
        8) y_label - the name of the OY axis of the graph
        9) filename - the relative path where the file will be saved (with the file name, the file extension is not required) or just the filename
        10) zoom_axes - the list containing the placement coordinates (x, y) and dimensions (width, height) of the smaller graph that contains the zoomed-in image for the specified limits (x_lims and y_lims)
            zoom_axes = [x, y, width, height] 
    """
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(df[x], df[y[0]], label='Raw Data', color='green')
    ax.plot(df[x], df[y[1]], label='Filtered_data', color='orange')
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
    plt.legend()

    # inset
    if zoom_axes:
        inset_ax = fig.add_axes(zoom_axes)
    else:
        inset_ax = fig.add_axes([0.25, 0.20, 0.20, 0.20])
    inset_ax.plot(df[x], df[y[0]], label='Raw Data', color='green')
    inset_ax.plot(df[x], df[y[1]], label='Filtered_data', color='orange')
    inset_ax.set_xlim(x_lims[0], x_lims[1])
    inset_ax.set_ylim(y_lims[0], y_lims[1])
    inset_ax.set_title('zoom near origin')
    if filename:
        plt.savefig(f"{filename}.png", bbox_inches='tight')
    plt.show()

In [ ]:
def get_statistical_measures_df(df, df_data_columns, functions, result_df_columns):
    """A function to create a dataframe that contains calculated statistical parameters for the specified columns of the original dataframe
    Args:
        1) df - the original dataframe that contains the required data
        2) df_data_columns - a list of column names of the original dataframe for which to find statistical measures
        3) functions - a list of references to numpy functions that will calculate the required statistical measures
        4) result_df_columns - a list of names of the columns of the output dataframe, in which the names of the searched statistical measures are specified
    
    An example of using the function:
        Let's imagine that we have a dataframe data_df, which contains the columns 'time' (time of measurement), 'accelX', 'accelY', 'accelZ' (the results of measuring the readings of the accelerometer on the corresponding axes). 
        We want to calculate the statistical mean and standard deviation for the accelerometer readings on all three axes (the columns 'accelX', 'accelY', 'accelZ' of data_df), so the columns of the output dataframe will be named, for example, 'mean' and 'std'. 
        These values can be calculated using the numpy library functions: np.mean and np.std.
        So, the function call will look like this:
        get_statistical_measures_df(df=data_df, 
                                    df_data_columns=['accelX', 'accelY', 'accelZ'], 
                                    functions=[np.mean, np.std], 
                                    result_df_columns=['mean', 'std'])
    """
    statistical_measures_dict = {}
    for key in result_df_columns:
        statistical_measures_dict[key] = []
        
    statistical_measures_dict['data'] = df_data_columns
    
    for [function, res_column] in zip(functions, result_df_columns):
        # Go through all df columns and find statistical measures for it
        for df_data_column in df_data_columns:
            statistical_measures_dict[res_column].append(function(df[df_data_column].values))
    
    # Create DataFrame
    statistical_measures_df = pd.DataFrame.from_dict(statistical_measures_dict)
    statistical_measures_df = statistical_measures_df.set_index('data')
    return statistical_measures_df

## Exploratory Data Analysis

In [ ]:
def display_pie_chart(df, column_name, title=None, filename=None):
    """ Function to display the percentage ratio of column (with the name column_name) content 
    Args:
        1) df - the original dataframe that contains the required information
        2) column_name - the name of the df dataframe column whose percentage values are to be found
        3) title - the title of the graph
        4) filename - the relative path where the file will be saved (with the file name, the file extension is not required) or just the filename
    Returns:
        None, but plots graph
    """
    # Calculate the percentage of each activity in original_df
    activity_percentages_df = df[column_name].value_counts(normalize=True) * 100

    plt.figure(figsize=(12, 6))

    # Plot pie chart for df
    sns.set_palette("Set3")
    plt.pie(activity_percentages_df, labels=activity_percentages_df.index, autopct='%1.1f%%', startangle=140)
    plt.title(title)

    # Adjust layout
    plt.tight_layout()
    if filename:
        plt.savefig(f'{filename}.png', bbox_inches='tight')
    plt.show()

## Windowing

In [ ]:
def windowed_pie_chart(original_df, windowed_df, column, first_chart_title='Original DataFrame', second_chart_title='Windowed DataFrame', filename=None):
    """Function for displaying the ratio of column content between the original dataframe and the windowed dataframe in the form of a pie chart
    Args:
        1) original_df - the original dataframe that contains the required information
        2) windowed_df - a dataframe that contains the results of windowing
        3) column - the name of the dataframe column whose percentage values are to be found
        4) filename - the relative path where the file will be saved (with the file name, the file extension is not required) or just the filename
    Returns:
        None; just builds a pie chart to display the ratio of column contents between the original dataframe and the windowed dataframe
    """
    # Calculate the percentage of each activity in original_df
    activity_percentages_df = original_df[column].value_counts(normalize=True) * 100

    # Calculate the percentage of each activity in windowed_df
    activity_percentages_windowed = windowed_df[column].value_counts(normalize=True) * 100

    # Create subplots for pie charts
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    # Plot pie chart for df
    sns.set_palette("Set3")
    axes[0].pie(activity_percentages_df, labels=activity_percentages_df.index, autopct='%1.1f%%', startangle=140)
    axes[0].set_title(first_chart_title)

    # Plot pie chart for windowed_df
    sns.set_palette("Set3")
    axes[1].pie(activity_percentages_windowed, labels=activity_percentages_df.index, autopct='%1.1f%%', startangle=140)
    axes[1].set_title(second_chart_title)

    # Adjust layout
    plt.tight_layout()
    if filename:
        plt.savefig(f'{filename}.png', bbox_inches='tight')
    plt.show()

## Feature Engineering

In [34]:
def get_statistical_measures_df(windowed_data_df, functions, data_df_columns, result_df_columns):
    """Function for creating a dataframe X_df, the columns of which correspond to the required statistical measures for the windows of the windowed_data_df dataframe
    Args:
        1) windowed_data_df - a dataframe whose rows contain arrays of data formed as a result of windowing
        2) functions - a list of references to lambda functions that will calculate the required statistical measures
        3) data_df_columns - a list of column names of the windowed_data_df dataframe for which to find statistical measures
        4) result_df_columns - a list of names of the searched statistical measures of the output dataframe
        
    Returns:
        X_train dataframe
        
    An example of using the function:
        Let's imagine we have a windowed_data_df dataframe (a dataframe formed as a result of windowing - that is, each record in a row can be an array of records for a given window), which contains the columns 'accX', 'accY', 'accZ' (the results of measuring the readings of the accelerometer on the corresponding axes). 
        Inside the function, we create an X_df dataframe to which we want to add new columns with statistical measure values.
        For example, we want to calculate the statistical mean and avg absolute diff for the accelerometer readings on all three axes (the columns 'accX', 'accY', 'accZ' of data_df), so the columns of the X_df dataframe will be named, for example, 'accX_mean', 'accY_mean', ..., 'accZ_aad'. 
        These values can be calculated using the lambda functions: [
            lambda x: x.mean(),
            lambda x: np.mean(np.absolute(x - np.mean(x)))
        ]
        So, the function call will look like this:
        get_statistical_measures_df(df=data_df, 
                                    functions=[
                                        lambda x: x.mean(),
                                        lambda x: np.mean(np.absolute(x - np.mean(x)))
                                    ], 
                                    data_df_columns=['accX', 'accY', 'accZ'], 
                                    result_df_columns=['mean', 'aad']) 
    """
    X_df = pd.DataFrame()
    for [function, res_column] in zip(functions, result_df_columns):
        for data_column in data_df_columns:
            X_df[f'{data_column}_{res_column}'] = windowed_data_df[data_column].apply(function)
    return X_df

# Work with train data

In [ ]:
df = pd.read_csv('data/40_Hz/concatenated_data/All/Train_activities_1_2023-08-23.csv')
df.head()

In [ ]:
df.info()

## Data Cleaning and Preprocessing

### We execute the following steps based on the observation:
- drop null values.
- drop the rows where the time is less than 0.
- sort data in ascending order of time if it has not been sorted.

In [ ]:
df.info()

In [ ]:
len(df[df['time']<0])

In [ ]:
# Check if the 'time' column of the dataframe is sorted in ascending order
df['time'].is_monotonic_increasing

## Exploring measurement period and frequency stability

In [ ]:
time_diffs = df['time'].diff()
freq = 1.0 / time_diffs.mean()
print(f"Measurement time = {df.iloc[-1]['time'] - df.iloc[0]['time']} s")
print(f"Number of measurements (number of rows in the data set) = {len(df)}")
print(f"Average measurement period = {time_diffs.mean():.3f} s")
print(f"Average frequency of measurement = {freq:.3f} Hz")

In [ ]:
time_measurement_df = get_measurement_time_df(df)
time_measurement_df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
ax.set_title(f'Stability of Data Collection (Raw Data)\nAverage frequency: {freq:.3f} Hz')
ax.set_ylabel('measurement time, s')
sns.lineplot(x=range(len(time_measurement_df['measurement_time'])), 
             y=time_measurement_df['measurement_time'])

inset_axes = fig.add_axes([0.17, 0.35, 0.10, 0.25])
inset_axes.plot(range(len(time_measurement_df['measurement_time'])), 
                time_measurement_df['measurement_time'])
inset_axes.set_title('Zoom near origin')
inset_axes.set_xlim(10, 20)
inset_axes.set_ylim(0.01, 0.04)
plt.savefig('graphs/train_data_collection_stability.png')
plt.show()

In [ ]:
time_measurement_df[time_measurement_df['measurement_time'] > 1]

### Delete rows with measurement_time > 0.040

In [ ]:
len(time_measurement_df[time_measurement_df['measurement_time'] <= 0.040])

In [ ]:
df = df[df['time'].diff() <= 0.04]
df.info()

In [ ]:
plt.figure(figsize=(12, 4))
plt.title(f'Stability of Data Collection (Raw Data)\nAverage frequency: {freq:.3f} Hz')
plt.ylabel('measurement time, s')
sns.lineplot(x=range(len(time_measurement_df[time_measurement_df['measurement_time'] <= 0.04]['measurement_time'])), 
             y=time_measurement_df[time_measurement_df['measurement_time'] <= 0.04]['measurement_time'])
plt.savefig('graphs/train_data_collection_stability.png')
plt.show()

## Data Filtering

### Display data before filtering

In [ ]:
display_three_axes(df=df, x='time', y=['accX', 'accY', 'accZ'], title='Time Dependence of Linear Acceleration (Raw Data)', 
                   x_label='Time, s', y_label='Linear acceleration, m/s^2', filename='graphs/accelerometer')

In [ ]:
display_three_axes(df=df, x='time', y=['gyrX', 'gyrY', 'gyrZ'], title='Time Dependence of Angular Velocity (Raw Data)', 
                   x_label='Time, s', y_label='Angular velocity, rad/s', filename='graphs/gyroscope')

### Display data after filtering

In [ ]:
window_size = 10  # Define the size of the median window

# Apply median filtering to accelerometer data
df['accX_filtered'] = df['accX'].rolling(window=window_size, center=True, min_periods=1).median()
df['accY_filtered'] = df['accY'].rolling(window=window_size, center=True, min_periods=1).median()
df['accZ_filtered'] = df['accZ'].rolling(window=window_size, center=True, min_periods=1).median()

# Apply median filtering to gyroscope data
df['gyrX_filtered'] = df['gyrX'].rolling(window=window_size, center=True, min_periods=1).median()
df['gyrY_filtered'] = df['gyrY'].rolling(window=window_size, center=True, min_periods=1).median()
df['gyrZ_filtered'] = df['gyrZ'].rolling(window=window_size, center=True, min_periods=1).median()

df.head(3)

In [ ]:
display_three_axes(df=df, x='time', y=['accX_filtered', 'accY_filtered', 'accZ_filtered'], title='Time Dependence of Linear Acceleration (Filtered Data)', 
                x_label='Time, s', y_label='Linear acceleration, m/s^2')

In [ ]:
display_three_axes(df=df, x='time', y=['gyrX_filtered', 'gyrY_filtered', 'gyrZ_filtered'], title='Time Dependence of Angular Velocity (Filtered Data)', 
                x_label='Time, s', y_label='Angular velocity, rad/s', filename='graphs/gyroscope')

### Consider the implications of data filtering

#### Display the results of filtering for accelerometer and gyroscope

##### Zoom accelerometer graphs to better see the results of filtering

In [ ]:
display_raw_filtered_data_zoom(df=df, x='time', y=['accX', 'accX_filtered'], x_lims=[14, 20], y_lims=[-1.5, 1.5],
                               x_label='Time, s', y_label='Linear acceleration, m/s^2',
                               title='Accelerometer OX: Raw vs Filtered',
                               zoom_axes = [0.18, 0.18, 0.15, 0.15],
                               filename='graphs/accX_filtered'
                              )

In [ ]:
display_raw_filtered_data_zoom(df=df, x='time', y=['accY', 'accY_filtered'], x_lims=[14, 20], y_lims=[-3, 3],
                               x_label='Time, s', y_label='Linear acceleration, m/s^2',
                               title='Accelerometer OX: Raw vs Filtered',
                               zoom_axes = [0.18, 0.18, 0.15, 0.15])

In [ ]:
display_raw_filtered_data_zoom(df=df, x='time', y=['accZ', 'accZ_filtered'], x_lims=[14, 20], y_lims=[4, 20],
                               x_label='Time, s', y_label='Linear acceleration, m/s^2',
                               title='gForce OX: Raw vs Filtered',
                               zoom_axes = [0.30, 0.65, 0.15, 0.15])

##### Zoom gyroscope graphs to better see the results of filtering

In [ ]:
display_raw_filtered_data_zoom(df=df, x='time', y=['gyrX', 'gyrX_filtered'], x_lims=[14, 20], y_lims = [-0.6, 1],
                               x_label='Time, s', y_label='Angular velocity, rad/s',
                               title='Gyroscope OX: Raw vs Filtered',
                               zoom_axes = [0.18, 0.18, 0.15, 0.15])

In [ ]:
display_raw_filtered_data_zoom(df=df, x='time', y=['gyrY', 'gyrY_filtered'], x_lims=[14, 20], y_lims = [-0.6, 0.75],
                               x_label='Time, s', y_label='Angular velocity, rad/s',
                               title='Gyroscope OY: Raw vs Filtered',
                               zoom_axes = [0.20, 0.50, 0.15, 0.15])

In [ ]:
display_raw_filtered_data_zoom(df=df, x='time', y=['gyrZ', 'gyrZ_filtered'], x_lims=[14, 20], y_lims = [-1, 1.2],
                               x_label='Time, s', y_label='Angular velocity, rad/s',
                               title='Gyroscope OZ: Raw vs Filtered',
                               zoom_axes = [0.18, 0.18, 0.15, 0.15],
                               filename='graphs/gyrZ_filtered'
                              )

## Exploratory Data Analysis

### Analyzing class label distribution

In [ ]:
sns.set_style('whitegrid')
plt.figure(figsize=(10, 5))
sns.countplot(data=df, x='activity')
plt.title('Number of samples by activity')
plt.show()

In [ ]:
display_pie_chart(df=df, column_name='activity', title='Class label distribution')

#### Remove 'No activity' class ('activity' column) from df 

In [ ]:
df.info()

In [ ]:
df = df[df['activity'] != 'No activity']
df.info()

In [ ]:
plt.rcdefaults()
sns.set_style('whitegrid')
plt.figure(figsize=(10, 5))
sns.countplot(data=df, x='activity')
plt.title('Number of samples by activity')
plt.show()

In [ ]:
activity_counts = df['activity'].value_counts()
print(activity_counts)

In [ ]:
display_pie_chart(df=df, column_name='activity', title='Class label distribution')

#### Perform undersampling to get a balanced dataframe

In [ ]:
# First, calculate the minimum number of samples across all classes
min_samples = df['activity'].value_counts().min()


# Initialize an empty DataFrame to store the undersampled data
undersampled_df = pd.DataFrame()

# Loop through each unique activity class and select the first min_samples for each class
for activity_class in df['activity'].unique():
    class_subset = df[df['activity'] == activity_class].iloc[:min_samples]
    undersampled_df = pd.concat([undersampled_df, class_subset])

undersampled_df.head()

In [ ]:
undersampled_df.reset_index(inplace=True)
undersampled_df.info()

#### View the labels distribution after undersampling

In [ ]:
activity_counts = undersampled_df['activity'].value_counts()
print(activity_counts)

In [ ]:
plt.rcdefaults()
sns.set_style('whitegrid')
sns.set_style('whitegrid')
plt.figure(figsize=(10, 5))
sns.countplot(data=undersampled_df, x='activity')
plt.title('Number of samples by activity')
plt.savefig('graphs/countplot.png')
plt.show()

In [ ]:
display_pie_chart(df=undersampled_df, column_name='activity', title='Class label distribution')

### Observe activity-wise distribution of the signal data along x, y and z axes

In [ ]:
undersampled_df.head()

#### Accelerometer

##### accX

In [ ]:
plt.rcdefaults()
sns.FacetGrid(data=undersampled_df, hue='activity', height=10).map(sns.distplot, 'accX_filtered').add_legend()
plt.grid(alpha=0.5)
plt.savefig('graphs/accX_distplot.png', bbox_inches='tight')

##### accY

In [ ]:
sns.FacetGrid(data=undersampled_df, hue='activity', height=10).map(sns.distplot, 'accY_filtered').add_legend()
plt.grid(alpha=0.5)
plt.savefig('graphs/accY_distplot.png', bbox_inches='tight')

##### accZ

In [ ]:
sns.FacetGrid(data=undersampled_df, hue='activity', height=10).map(sns.distplot, 'accZ_filtered').add_legend()
plt.grid(alpha=0.5)
plt.savefig('graphs/accZ_distplot.png', bbox_inches='tight')

#### Gyroscope

##### gyrX

In [ ]:
sns.FacetGrid(data=undersampled_df, hue='activity', height=10).map(sns.distplot, 'gyrX_filtered').add_legend()
plt.grid(alpha=0.5)
plt.savefig('graphs/gyrX_distplot.png', bbox_inches='tight')

##### gyrY

In [ ]:
sns.FacetGrid(data=undersampled_df, hue='activity', height=10).map(sns.distplot, 'gyrY_filtered').add_legend()
plt.grid(alpha=0.5)
plt.savefig('graphs/gyrY_distplot.png', bbox_inches='tight')

##### gyrZ

In [ ]:
sns.FacetGrid(data=undersampled_df, hue='activity', height=10).map(sns.distplot, 'gyrZ_filtered').add_legend()
plt.grid(alpha=0.5)
plt.savefig('graphs/gyrZ_distplot.png', bbox_inches='tight')

### Build a correlation matrix and remove certain axes of the accelerometer or gyroscope

In [ ]:
undersampled_df.drop('index', axis=1, inplace=True)
undersampled_df.head()

In [ ]:
sel_columns = ['time', 'accX_filtered', 'accY_filtered', 'accZ_filtered', 'gyrX_filtered', 'gyrY_filtered', 'gyrZ_filtered']

# Extract the selected columns from the DataFrame
selected_data = undersampled_df[sel_columns]

# Calculate the correlation matrix for the selected columns
corr_matrix = selected_data.corr()

# Create a heatmap using seaborn
plt.figure(figsize=(10, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Matrix Heatmap')
plt.savefig('graphs/correlation_matrix.png')
plt.show()

In [ ]:
filtered_columns = ['time', 'accX_filtered', 'accY_filtered', 'accZ_filtered', 'gyrZ_filtered', 'activity']
filtered_df = undersampled_df[filtered_columns]
filtered_df.head()

In [ ]:
filtered_df.info()

## Data Transformation

In [ ]:
# Calculate the number of data points within a 2-second window
sampling_frequency =  1.0 / filtered_df['time'].diff().mean()  # Hz
window_duration = 2 # seconds
window_size = ceil(sampling_frequency * window_duration)
step_size = window_size // 2

# Create a list to store the windowed dataframes
windowed_dfs = []

print(f"window size = {window_size}")

windowed_dict = {'start_time': [], 'end_time': [], 'accX': [], 'accY': [], 'accZ': [], 'gyrZ': [], 'activity': []}
# Divide the entire dataframe into 2-second windows
for i in range(0, len(filtered_df), step_size):
    window_df = filtered_df.iloc[i:i + window_size]
    
    windowed_dict['start_time'].append(window_df['time'].iloc[0])
    windowed_dict['end_time'].append(window_df['time'].iloc[-1])
    windowed_dict['accX'].append(window_df['accX_filtered'].values)
    windowed_dict['accY'].append(window_df['accY_filtered'].values)
    windowed_dict['accZ'].append(window_df['accZ_filtered'].values)
    
    windowed_dict['gyrZ'].append(window_df['gyrZ_filtered'].values)
    
    # Determine the most frequent activity in the window
    most_frequent_activity = window_df['activity'].value_counts().idxmax()
    label = stats.mode(filtered_df['activity'][i: i + window_size])[0][0]
    if label != most_frequent_activity:
        print("label != most_frequent_activity")
        print(f'most_frequent_activity = {most_frequent_activity}')
        print(f'label = {label}')
        break
    # Assign the most frequent activity to all rows in the window
    windowed_dict['activity'].append(most_frequent_activity)
    
    windowed_dfs.append(window_df)

windowed_df = pd.DataFrame.from_dict(windowed_dict)
print(f"len(windowed_dfs) = {len(windowed_dfs)}")
print(f"len(windowed_df) = {len(windowed_df)}")

In [ ]:
sampling_frequency =  1.0 / filtered_df['time'].diff().mean()  # Hz
print(f"sampling_frequency = {sampling_frequency: .3f} Hz")

In [ ]:
windowed_df.head()

In [ ]:
windowed_df.info()

In [ ]:
windowed_df[-1:]

In [ ]:
print(f'len(filtered_df) = {len(filtered_df)}')
print(f'len(windowed_df) = {len(windowed_df)}')

### View the results of windowing

In [ ]:
sns.set_palette("Set3")
windowed_pie_chart(original_df=filtered_df, windowed_df=windowed_df, column='activity')

## Feature Engineering

#### In the stage 1 of the feature engineering, we will build a total of 12 simple statistical features
1. mean
2. standard deviation
3. average absolute deviation
4. minimum value
5. maximum value
6. difference of maximum and minimum values
7. median
8. interquartile range
9. negative values count
10. positive values count
11. skewness = assymetry
12. kurtosis

In [28]:
windowed_df.head(2)

,start_time,end_time,accX,accY,accZ,gyrZ,activity
0,4.774,6.697,"[0.054000003000000005, 0.065025002, 0.06502500...","[-0.265500015, -0.265500015, -0.265500015, -0....","[9.780525, 9.644025500000001, 9.56400100000000...","[0.00680625, -0.03911875, -0.0719125, -0.07191...",Squat
1,5.776,7.683,"[0.119025005, 0.119025005, 0.0785250065, 0.002...","[0.284475015, 0.55350001, 0.766500025, 0.76650...","[11.6035505, 11.6300255, 11.749500000000001, 1...","[0.14403125, 0.127875, 0.127875, 0.05218125, -...",Squat


In [29]:
functions_list = [
    lambda x: x.mean(), # mean
    lambda x: x.std(),  # std deviation 
    lambda x: np.mean(np.absolute(x - np.mean(x))), # avg absolute diff
    lambda x: x.min(), # min
    lambda x: x.max(), # max
    lambda x: x.max() - x.min(), # range = max-min diff
    lambda x: np.median(x), # median
    lambda x: np.percentile(x, 75) - np.percentile(x, 25), # interquartile range
    lambda x: np.sum(x < 0), # negative count
    lambda x: np.sum(x > 0), # positive count
    lambda x: stats.skew(x), # skewness = assymetry
    lambda x: stats.kurtosis(x) # kurtosis
]

In [37]:
X_train = get_statistical_measures_df(windowed_data_df=windowed_df, 
                                      functions=functions_list, 
                                      data_df_columns=['accX', 'accY', 'accZ', 'gyrZ'], 
                                      result_df_columns=['mean', 'std', 'aad', 'min', 'max', 'range', 'median', 'iqr', 'neg_count', 'pos_count', 'assymetry', 'kurtosis'])

y_train = windowed_df['activity'].values
X_train.head()

,accX_mean,accY_mean,accZ_mean,gyrZ_mean,accX_std,accY_std,accZ_std,gyrZ_std,accX_aad,accY_aad,...,accZ_pos_count,gyrZ_pos_count,accX_assymetry,accY_assymetry,accZ_assymetry,gyrZ_assymetry,accX_kurtosis,accY_kurtosis,accZ_kurtosis,gyrZ_kurtosis
0,0.204513,0.043750,10.181814,-0.041990,0.191702,0.446751,2.493435,0.173847,0.164112,0.352874,...,62,25,0.889454,0.846221,0.324005,-0.531108,-0.783717,-0.403698,-0.816060,-0.840034
1,0.339821,0.349124,10.100784,-0.048229,0.211233,0.441290,2.552319,0.316891,0.181807,0.378584,...,62,26,-0.752718,-0.129656,0.268890,0.523141,-0.661552,-1.095923,-0.773074,-0.904146
2,0.316942,0.815667,9.721920,-0.079716,0.216026,0.908844,2.554711,0.334650,0.187870,0.699350,...,62,25,-0.348733,0.969841,0.788509,0.564838,-0.898058,0.030919,-0.327959,-0.866246
3,0.193883,0.900957,9.749154,-0.026190,0.288013,0.857635,3.219180,0.307127,0.219851,0.704377,...,62,30,-0.037762,1.113915,0.474904,0.199509,0.485459,-0.065963,-1.263340,-1.162199
4,0.037838,0.780090,9.024925,0.038186,0.278074,0.454869,2.237759,0.241641,0.218381,0.408583,...,62,31,0.504361,-0.236346,0.917411,0.420765,1.228187,-1.453429,0.955438,-0.839217


In [38]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1799 entries, 0 to 1798
Data columns (total 48 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   accX_mean       1799 non-null   float64
 1   accY_mean       1799 non-null   float64
 2   accZ_mean       1799 non-null   float64
 3   gyrZ_mean       1799 non-null   float64
 4   accX_std        1799 non-null   float64
 5   accY_std        1799 non-null   float64
 6   accZ_std        1799 non-null   float64
 7   gyrZ_std        1799 non-null   float64
 8   accX_aad        1799 non-null   float64
 9   accY_aad        1799 non-null   float64
 10  accZ_aad        1799 non-null   float64
 11  gyrZ_aad        1799 non-null   float64
 12  accX_min        1799 non-null   float64
 13  accY_min        1799 non-null   float64
 14  accZ_min        1799 non-null   float64
 15  gyrZ_min        1799 non-null   float64
 16  accX_max        1799 non-null   float64
 17  accY_max        1799 non-null   f

In [39]:
y_train[:5]

array(['Squat', 'Squat', 'Squat', 'Squat', 'Squat'], dtype=object)

In [40]:
print(f"len(X_train) = {len(X_train)}")
print(f"len(y_train) = {len(y_train)}")

len(X_train) = 1799
len(y_train) = 1799


# Work with test data

In [ ]:
test_df = pd.read_csv('data/40_Hz/concatenated_data/All/Test_activities_1_2023-08-23.csv')
test_df.head()

In [ ]:
test_df.info()

## Exploring measurement period and frequency stability

In [ ]:
time_diffs = test_df['time'].diff()
freq = 1.0 / time_diffs.mean()
print(f"Measurement time = {test_df.iloc[-1]['time'] - test_df.iloc[0]['time']} s")
print(f"Number of measurements (number of rows in the data set) = {len(test_df)}")
print(f"Average measurement period = {time_diffs.mean():.3f} s")
print(f"Average frequency of measurement = {freq:.3f} Hz")

In [ ]:
time_measurement_df = get_measurement_time_df(test_df)
time_measurement_df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
ax.set_title(f'Stability of Data Collection (Raw Data)\nAverage frequency: {freq:.3f} Hz')
ax.set_ylabel('measurement time, s')
sns.lineplot(x=range(len(time_measurement_df['measurement_time'])), 
             y=time_measurement_df['measurement_time'])
plt.savefig('graphs/train_data_collection_stability.png')
plt.show()

In [ ]:
len(time_measurement_df[time_measurement_df['measurement_time'] <= 0.040])

In [ ]:
len(test_df[test_df['time'].diff() > 0.040])